In [1]:
import sys, os
import pandas as pd
import numpy as np

# Ensure project root is on path
sys.path.append("/Users/lei/Documents/Sequenzo_all_folders/glm_plus")

from glm_plus.ordinal_quantile_regression.ori import quantregOR1
from glm_plus.ordinal_quantile_regression.orii import quantregOR2
from glm_plus.ordinal_quantile_regression.panel_oqr import (
    build_panel_design, fit_panel_oqr,
    extract_gender_year_effects, probability_top_greater_bottom,
    plot_gender_gap_trends,
)

In [2]:
df_seniority = pd.read_csv("/Users/lei/Documents/Sequenzo_all_folders/sequenzo_local/test_data/real_data_my_paper/detailed_senority_levels_for_10_years.csv")
df_seniority

,worker_id,gender,country,cohort,highest_educational_degree,whether_bachelor_university_prestigious,internationalization,work_years,company_size,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10
0,dilip-kumar-nath-7538746a,male,india,11-20 total work years,Bachelor,False,Multinational,15.250,"1,001-5,000 employees",Regular,Regular,Regular,Regular,Senior,Senior,Senior,Senior,Senior,Senior
1,vinodkumar-yadav-b7a976107,male,india,11-20 total work years,Bachelor,False,Local,11.500,"5,001-10,000 employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
2,rajani-kulkarni-72674649,female,india,11-20 total work years,Bachelor,False,Multinational,11.250,"10,001+ employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Leader,Leader,Regular
3,josh-slosson-07a1509,male,india,11-20 total work years,Bachelor,False,International,18.417,11-50 employees,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
4,jonatthan-rodriguez-8a5612121,male,india,11-20 total work years,Bachelor,False,Multinational,11.917,"5,001-10,000 employees",Regular,Regular,Senior,Senior,Senior,Senior,Senior,Senior,Senior,Senior
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32623,edwardgu1915,male,us,0-10 total work years,Master,False,Multinational,10.499,"501-1,000 employees",Leader,Leader,Leader,Leader,Leader,Senior,Senior,Senior,Regular,Senior
32624,terry-ferguson-8060363a,male,us,21-30 total work years,Bachelor,False,Local,21.001,"10,001+ employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
32625,christy-garner-677894213,female,us,21-30 total work years,Bachelor,False,Multinational,27.250,"501-1,000 employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
32626,jay-p-119369202,male,us,21-30 total work years,Bachelor,False,Local,23.417,"10,001+ employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular


In [3]:
# Wide-format cleaning: collapse seniority labels into 4 levels
# - Assistant + Junior → Assistant/Junior
# - Regular → Regular
# - Senior/Leader/Lead → Leader
# - Chief or Founder (+ VP/Director/C-level variants) → Chief/Founder

# Detect Y columns
Y_cols = [c for c in df_seniority.columns if str(c).upper().startswith('Y')]
assert len(Y_cols) > 0, 'No Y1..Yk columns found'

collapse_order = ["Assistant/Junior", "Regular", "Leader", "Chief/Founder"]

def _collapse_label(v):
    if pd.isna(v):
        return np.nan
    t = str(v).strip().lower()
    if t in {"assistant", "intern", "entry level"}:
        return "Assistant/Junior"
    if t in {"junior", "jr"}:
        return "Assistant/Junior"
    if t in {"regular", "mid", "middle", "intermediate", "associate"}:
        return "Regular"
    if t in {"senior", "leader", "lead", "sr"}:
        return "Leader"
    if t in {"chief or founder", "chief/founder", "chief", "c-level", "founder", "executive", "vp", "vice president", "director"}:
        return "Chief/Founder"
    # keep original if already one of target labels
    if v in collapse_order:
        return v
    return v

for col in Y_cols:
    df_seniority[col] = df_seniority[col].apply(_collapse_label)

# Quick check of levels after collapsing
level_counts = pd.DataFrame({c: df_seniority[c].value_counts(dropna=False) for c in Y_cols})
print(level_counts.sum(axis=1).sort_index())

df_seniority.head()


Assistant/Junior     11110
Chief/Founder         2504
Leader              114640
Regular             198026
dtype: int64


,worker_id,gender,country,cohort,highest_educational_degree,whether_bachelor_university_prestigious,internationalization,work_years,company_size,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10
0,dilip-kumar-nath-7538746a,male,india,11-20 total work years,Bachelor,False,Multinational,15.250,"1,001-5,000 employees",Regular,Regular,Regular,Regular,Leader,Leader,Leader,Leader,Leader,Leader
1,vinodkumar-yadav-b7a976107,male,india,11-20 total work years,Bachelor,False,Local,11.500,"5,001-10,000 employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
2,rajani-kulkarni-72674649,female,india,11-20 total work years,Bachelor,False,Multinational,11.250,"10,001+ employees",Regular,Regular,Regular,Regular,Regular,Regular,Regular,Leader,Leader,Regular
3,josh-slosson-07a1509,male,india,11-20 total work years,Bachelor,False,International,18.417,11-50 employees,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular,Regular
4,jonatthan-rodriguez-8a5612121,male,india,11-20 total work years,Bachelor,False,Multinational,11.917,"5,001-10,000 employees",Regular,Regular,Leader,Leader,Leader,Leader,Leader,Leader,Leader,Leader


In [4]:
# 1) Wide → Long transformation (use already-collapsed labels)
# Assumptions:
# - Y1..Yk columns have been cleaned to 4 labels in the previous cell
# - We reshape and map those labels to integers 1..4

# Identify Y columns automatically (already defined above)
Y_cols = [c for c in df_seniority.columns if c.upper().startswith('Y')]
assert len(Y_cols) > 0, "No Y1..Yk columns found"

# Keep id and covariates
id_col = 'worker_id'
year_map = {c: int(c[1:]) for c in Y_cols}  # Y1 -> 1, Y10 -> 10

long_df = (
    df_seniority
    .melt(id_vars=[c for c in df_seniority.columns if c not in Y_cols],
          value_vars=Y_cols,
          var_name='Y', value_name='seniority_label')
    .assign(year=lambda d: d['Y'].map(year_map))
    .drop(columns=['Y'])
)

# Map 4 clean labels to ordinal integers
label_to_int = {"Assistant/Junior": 1, "Regular": 2, "Leader": 3, "Chief/Founder": 4}
long_df['seniority'] = long_df['seniority_label'].map(label_to_int)

# Gender to binary 0/1 with 1=female
gender_map = {
    'female': 1, 'Female': 1, 'F': 1, 1: 1,
    'male': 0, 'Male': 0, 'M': 0, 0: 0
}
long_df['gender01'] = long_df['gender'].map(gender_map)

# Drop rows with missing seniority or gender
long_df = long_df.dropna(subset=['seniority', 'gender01']).copy()
long_df['seniority'] = long_df['seniority'].astype(int)
long_df['gender01'] = long_df['gender01'].astype(int)

# Basic sanity checks
J = long_df['seniority'].nunique()
print(f"Long panel shape: {long_df.shape}, J={J}")
print(long_df['seniority_label'].value_counts(dropna=False).to_dict())
long_df.head()


Long panel shape: (326280, 13), J=4
{'Regular': 198026, 'Leader': 114640, 'Assistant/Junior': 11110, 'Chief/Founder': 2504}


,worker_id,gender,country,cohort,highest_educational_degree,whether_bachelor_university_prestigious,internationalization,work_years,company_size,seniority_label,year,seniority,gender01
0,dilip-kumar-nath-7538746a,male,india,11-20 total work years,Bachelor,False,Multinational,15.250,"1,001-5,000 employees",Regular,1,2,0
1,vinodkumar-yadav-b7a976107,male,india,11-20 total work years,Bachelor,False,Local,11.500,"5,001-10,000 employees",Regular,1,2,0
2,rajani-kulkarni-72674649,female,india,11-20 total work years,Bachelor,False,Multinational,11.250,"10,001+ employees",Regular,1,2,1
3,josh-slosson-07a1509,male,india,11-20 total work years,Bachelor,False,International,18.417,11-50 employees,Regular,1,2,0
4,jonatthan-rodriguez-8a5612121,male,india,11-20 total work years,Bachelor,False,Multinational,11.917,"5,001-10,000 employees",Regular,1,2,0


In [5]:
# 2) Cross-sectional OQR using wide format (faster)
# Use wide format with latest year for each person: x = [1, gender]
# Speed advantage: n_people vs n_people × n_years

# Choose strategy for multiple years per person:
# - 'last': use most recent non-null year
# - 'first': use earliest non-null year  
# - 'mode': use most common seniority level across years
strategy = 'last'

def _get_representative_seniority(row):
    Y_vals = [row[f'Y{i}'] for i in range(1, 11) if f'Y{i}' in df_seniority.columns]
    Y_vals = [v for v in Y_vals if pd.notna(v)]
    if not Y_vals:
        return np.nan
    if strategy == 'last':
        return Y_vals[-1]
    elif strategy == 'first':
        return Y_vals[0]
    elif strategy == 'mode':
        from collections import Counter
        return Counter(Y_vals).most_common(1)[0][0]
    return Y_vals[-1]  # fallback

df_wide_cs = df_seniority.copy()
df_wide_cs['seniority_rep'] = df_wide_cs.apply(_get_representative_seniority, axis=1)

# Map to integers
label_to_int = {"Assistant/Junior": 1, "Regular": 2, "Leader": 3, "Chief/Founder": 4}
df_wide_cs['seniority'] = df_wide_cs['seniority_rep'].map(label_to_int)

# Gender encoding
gender_map = {'female': 1, 'Female': 1, 'F': 1, 1: 1, 'male': 0, 'Male': 0, 'M': 0, 0: 0}
df_wide_cs['gender01'] = df_wide_cs['gender'].map(gender_map)

# Drop missing
df_wide_cs = df_wide_cs.dropna(subset=['seniority', 'gender01', 'country'])

countries_wide = df_wide_cs['country'].value_counts().index.tolist()

# Parallelized per-country estimation (falls back to sequential if joblib missing)
try:
    from joblib import Parallel, delayed
    n_jobs = max(1, min(8, (os.cpu_count() or 2) - 1)) if 'os' in globals() else -1
    use_parallel = True
except Exception:
    use_parallel = False


def _run_country(c):
    sub = df_wide_cs[df_wide_cs['country'] == c]
    if sub.empty or sub['seniority'].nunique() < 3:
        return None

    x_cs = np.column_stack([
        np.ones(len(sub)),
        sub['gender01'].to_numpy(dtype=float),
    ])
    y_cs = sub['seniority'].to_numpy(dtype=int).reshape(-1, 1)

    Jc = np.unique(y_cs).size
    k_cs = x_cs.shape[1]
    b0 = np.zeros((k_cs, 1))
    B0 = 10.0 * np.eye(k_cs)
    d0 = np.zeros((Jc - 2, 1))
    D0 = 0.25 * np.eye(Jc - 2)

    res_bot = quantregOR1(y=y_cs, x=x_cs, b0=b0, B0=B0, d0=d0, D0=D0,
                          burn=400, mcmc=1600, p=0.2, tune=0.1, verbose=False)
    res_top = quantregOR1(y=y_cs, x=x_cs, b0=b0, B0=B0, d0=d0, D0=D0,
                          burn=400, mcmc=1600, p=0.8, tune=0.1, verbose=False)

    bottom_effect = res_bot['betadraws'][1, :]
    top_effect = res_top['betadraws'][1, :]
    p_top_gt_bottom = float(np.mean((top_effect - bottom_effect) > 0.0))

    return {
        'country': c,
        'n_obs': int(len(sub)),
        'J': int(Jc),
        'bottom_mean': float(np.mean(bottom_effect)),
        'top_mean': float(np.mean(top_effect)),
        'P(top>bottom)': p_top_gt_bottom,
    }

if use_parallel:
    results_wide = Parallel(n_jobs=n_jobs, prefer='processes')(
        delayed(_run_country)(c) for c in countries_wide
    )
    results_wide = [r for r in results_wide if r is not None]
else:
    results_wide = []
    for c in countries_wide:
        r = _run_country(c)
        if r is not None:
            results_wide.append(r)

cs_wide_by_country = pd.DataFrame(results_wide).sort_values('P(top>bottom)', ascending=False)
print(f"Wide format analysis (strategy: {strategy})")
cs_wide_by_country


OverflowError: math range error

In [ ]:
# Cross-sectional visualization: sticky floor vs glass ceiling by country
try:
    import matplotlib.pyplot as plt
    
    # Use wide format results (faster version)
    df_plot = cs_wide_by_country.copy()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: P(top>bottom) by country (horizontal bar)
    colors = ['red' if p < 0.5 else 'blue' for p in df_plot['P(top>bottom)']]
    ax1.barh(df_plot['country'], df_plot['P(top>bottom)'], color=colors, alpha=0.7)
    ax1.axvline(0.5, color='black', linestyle='--', alpha=0.5)
    ax1.set_xlabel('P(top > bottom)')
    ax1.set_title('Glass Ceiling vs Sticky Floor by Country')
    ax1.text(0.1, len(df_plot)-0.5, 'Sticky Floor\n(red)', fontsize=9, ha='left')
    ax1.text(0.9, len(df_plot)-0.5, 'Glass Ceiling\n(blue)', fontsize=9, ha='right')
    
    # Plot 2: bottom vs top effect scatter
    ax2.scatter(df_plot['bottom_mean'], df_plot['top_mean'], 
               c=df_plot['P(top>bottom)'], cmap='RdBu_r', s=60, alpha=0.8)
    ax2.axhline(0, color='gray', linestyle='-', alpha=0.3)
    ax2.axvline(0, color='gray', linestyle='-', alpha=0.3)
    ax2.plot([-0.5, 0.5], [-0.5, 0.5], 'k--', alpha=0.3)  # diagonal
    ax2.set_xlabel('Bottom quantile gender effect (p=0.2)')
    ax2.set_ylabel('Top quantile gender effect (p=0.8)')
    ax2.set_title('Bottom vs Top Gender Effects')
    
    # Add country labels
    for i, row in df_plot.iterrows():
        ax2.annotate(row['country'], (row['bottom_mean'], row['top_mean']), 
                    xytext=(5, 5), textcoords='offset points', fontsize=8, alpha=0.8)
    
    plt.tight_layout()
    plt.show()
    
    # Summary interpretation
    sticky_countries = df_plot[df_plot['P(top>bottom)'] < 0.5]['country'].tolist()
    glass_countries = df_plot[df_plot['P(top>bottom)'] > 0.5]['country'].tolist()
    
    print(f"\nInterpretation:")
    print(f"Sticky Floor countries (P<0.5): {sticky_countries}")
    print(f"Glass Ceiling countries (P>0.5): {glass_countries}")
    
except Exception as e:
    print('Cross-sectional visualization skipped:', e)
    print("Use table results directly:")
    print(cs_wide_by_country)


In [ ]:
# 3) Panel OQR by country (year FE + gender×year interactions)
# For each country, build its own design and fit at p in {0.2, 0.8}

panel_results = []
per_country_tables = {}
for c in countries:
    sub = long_df[long_df['country'] == c]
    if sub.empty:
        continue
    if sub['seniority'].nunique() < 3:
        continue

    # Build design for this country
    sub_for_design = sub.rename(columns={'gender01': 'gender'})
    controls = [
        col for col in [
            'highest_educational_degree',
            'whether_bachelor_university_prestigious',
            'internationalization',
            'work_years',
            'company_size',
            'cohort'
        ] if col in sub_for_design.columns
    ]
    design_c = build_panel_design(
        df=sub_for_design,
        outcome_col='seniority',
        year_col='year',
        gender_col='gender',
        controls=controls,
        cohort_col='cohort' if 'cohort' in sub_for_design.columns else None,
        drop_first_year=True,
        gender_is_female_one=True,
    )

    fit_c = fit_panel_oqr(
        design_c,
        burn=800,
        mcmc=3200,
        quantiles=(0.2, 0.8),
        verbose=False,
    )

    p_overall, per_year = probability_top_greater_bottom(
        fit_c, p_top=0.8, p_bottom=0.2, aggregate=True
    )
    panel_results.append({
        'country': c,
        'n_obs': int(len(sub)),
        'P(top>bottom)': float(p_overall),
    })
    per_country_tables[c] = per_year

panel_by_country = pd.DataFrame(panel_results).sort_values('P(top>bottom)', ascending=False).reset_index(drop=True)
panel_by_country


In [ ]:
# 4) Visualization: time trends for sticky floor vs glass ceiling
try:
    import matplotlib.pyplot as plt
    ax = plot_gender_gap_trends(
        fit, quantiles=(0.2, 0.5, 0.8),
        title='Female − Male seniority readiness gap over time'
    )
    plt.show()
except Exception as e:
    print('Plotting skipped (matplotlib not available or other error):', e)
